In [0]:
import pandas as pd
import os
import sys
import numpy as np
import scipy as sp
import pickle
from IPython.display import FileLink

In [0]:
translation_df = pd.read_csv("translation_table.csv", sep="\t", index_col=0)
def get_translated_name(n):
    row = translation_df.ix[n.strip()]
    return "%s_%d_%d" % (row['pop'], row.indiv, row.dup)

In [0]:
translation_df

In [0]:
with open("/gpfs_fs/home/eckertlab/gypsy_indiv/raw_demult/bowtie_results.pkl", "rb") as r:
    bowtie_results = pickle.load(r)

In [0]:
overall = []
pop_mapping = {}
pop_reads = {}
for r in bowtie_results:
    sam = r[1]
    sample = os.path.basename(sam).split(".")[0]
    name = get_translated_name(sample)
    popn = name.split("_")[0]
    res = r[-1]
    perc = float(res[-1].split("%")[0])
    reads = float(res[0].split()[0])
    overall.append(perc)
    if not popn in pop_mapping:
        pop_mapping[popn] = []
        pop_reads[popn] = []
    pop_mapping[popn].append(perc)
    pop_reads[popn].append(reads)

In [0]:
pd.Series(overall).describe()

In [0]:
overall_mapping = {}
overall_reads = {}
for p in pop_mapping:
    print(p)
    overall_mapping[p] = pd.Series(pop_mapping[p]).describe()
    overall_reads[p] = pd.Series(pop_reads[p]).describe()

In [0]:
mapping_df = pd.DataFrame(overall_mapping).T.sort_index()
reads_df = pd.DataFrame(overall_reads).T.sort_index()

In [0]:
mapping_df

In [0]:
reads_df

In [0]:
stats = reads_df.join(mapping_df, lsuffix="_reads", rsuffix="_mapping")

In [0]:
stats

In [0]:
out = "fastq_stats.xlsx"
stats.to_excel(out)
FileLink(out)